# Tutorial 1

## other doc sources
https://github.com/MarcToussaint/rai-maintenance/tree/master/help

In [1]:
import sys
sys.path.append('../src/ry')
from libry import *
from numpy import *

The starting point is to create a `Configuration`.

In [2]:
K = Configuration()

Other computational modules -- such as computing views, path planning, inverse kinematics, physical simulation, collision detection -- all refer to a particular configuration. You get a handle to these algorithms from a configuration. Here, we get a view for configuration `K`. (Recommendation: make the view window appear "Always On Top")

In [3]:
D = K.camera()

You can add things (objects, scene models, robots) to a configuration. As the view is strictly referring to the configuration, it automatically updates.

In [4]:
K.clear();
K.addFile('../rai-robotModels/pr2/pr2.g');
K.addFile('../test/kitchen.g');

TODO: We can create a 2nd configuration, and copy it.

In [5]:
#K2 = K
#D2 = K2.camera()
#q = K2.getJointState()
#q = 0*q
#K2.setJointState(q)

A configuration is a tree of frames. Some frames have a fixed relative transformation to the parent, others are joints, meaning that the relative transformation to the parent is determined by *degrees of freedom*. We can query and set the *joint state*, which is vector of all degrees of freedom:

In [6]:
q = K.getJointState()
print('joint state: ', q)
print('joint names: ', K.getJointNames() )
q[2] = q[2] + 1.
K.setJointState(q)

joint state:  [ 0.    0.    0.    0.1   0.    0.   -1.    1.    0.4   0.5   0.5  -1.    1.
 -2.   -2.   -1.5   1.5  -0.5  -0.5  -0.5   0.5   0.1   0.1   0.01  0.01]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'torso_lift_joint', 'head_pan_joint', 'laser_tilt_mount_joint', 'r_shoulder_pan_joint', 'l_shoulder_pan_joint', 'head_tilt_joint', 'r_shoulder_lift_joint', 'l_shoulder_lift_joint', 'r_upper_arm_roll_joint', 'l_upper_arm_roll_joint', 'r_elbow_flex_joint', 'l_elbow_flex_joint', 'r_forearm_roll_joint', 'l_forearm_roll_joint', 'r_wrist_flex_joint', 'l_wrist_flex_joint', 'r_wrist_roll_joint', 'l_wrist_roll_joint', 'r_gripper_l_finger_joint', 'l_gripper_l_finger_joint', 'r_gripper_joint', 'l_gripper_joint']


The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], wich position p and quaternion q.

In [8]:
X0 = K.getFrameState()
print('frame state: ', X0)
X = X0 + .1
K.setFrameState(X.flatten().tolist())

frame state:  [[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.47942554]
 [-0.02701518 -0.04207355  0.89067506 ...,  0.33900515 -0.62054477
   0.33900515]
 ..., 
 [-2.          0.          0.385      ...,  0.          0.          0.707107  ]
 [-0.5         2.          0.4        ...,  0.          0.          0.        ]
 [ 0.5         2.          0.4        ...,  0.          0.          0.        ]]


What happened here? We brutally added 0.1 to all poses of all frames, also making the quaternions non-normalized. This is setting frame poses in world coordinates; internally all the relative poses of the frame tree are computed as consistent to the absolute frames (child->X/parent->X). If a frame happens to be a joint, also the respective joint state is computed, e.g., reading out what rotation angle around the joint axis these absolute frames imply. If the set absolute frames are inconsistent to joint constraints that leaves the configuration in an awkward state: For now, the absolute frames are as commanded. But if we'd read out the joint state and set the same joint state, the frame state will change:

In [9]:
K.setJointState( K.getJointState() )

Let's bring the configuration back into the state before the harsh *setFrame*

In [10]:
K.setFrameState(X0.flatten().tolist())

We can add new frames. First a yellow ball (without parent) at absolute pose `X`:

In [11]:
K.addFrame("ball", "", "shape:sphere size:[0 0 0 .1] color:[1 1 0] X:<t(.8 .8 1.5)>" );

Or a sphere-swept box attached to the left hand with relative pose `Q`:

In [12]:
K.addFrame("hand", "pr2L", "shape:ssBox size:[.3 .2 .1 .01] color:[1 1 0] Q:<t(0 0 0)>" );

We can also add a frame, attached to the head, which has no shape associated to it, but create a view is associated with that frame:

In [13]:
K.addFrame("camera", "head_tilt_link", "Q:<d(-90 1 0 0) d(180 0 0 1)> focalLength:.3")
C = K.camera(frame="camera")

When we move the robot, that view moves with it:

In [14]:
K.setJointState(q=asarray([1.]), joints=['head_pan_joint'])

To close a view (or destroy a handle to a computational module), we reassign it to zero. We can also remove a frame from the configuration.

In [15]:
C = 0
K.delFrame('camera')

This solves a simple IK problem, defined by an equality constraint on the difference in position of 'ball' and 'hand'

In [16]:
IK = K.komo_IK()
IK.addObjective(type='eq', feature='positionDiff', frames=['ball', 'hand'])
IK.optimize()

We can reuse the optimization object, change the objective a bit (now the position difference is constrained to be [.1,.1,.1] in world coordinates), and reoptimize

In [17]:
IK.clearObjectives()
IK.addObjective(type='eq', feature='positionDiff', frames=['hand', 'ball'], target=[.1, .1, .1])
IK.optimize()

TODO demos:

* rename Camera -> View
* copy configurations
* have multiple configurations and views in parallel
* selecting/modifying DOFs (i.e., which joints are considered DOFs)
* I/O with other file formats?